In [4]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [5]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '438'
start = '2019-04-01T00:00:00Z'
end = '2021-05-21T00:00:00Z'
mongo_org_collection = db[str(organization_id)]

In [6]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': "web",
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.custom_fields": { "id": 21726556, "value": "ekahau_analyzer" }
    })
    print(tickets.count())
    file_path = os.path.expanduser('~/Downloads/ek_products.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket['original_item']
            j = {
                "text_a": i["subject"],
                "text_b": i["description"],
                "created_at": i["created_at"],
                "product": i["custom_fields"]
            }
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count > 99:
                break
    print("total: ", count)


get_dataset()

<ipython-input-6-05358d6bf1ed>:9: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(tickets.count())
91
total:  91
